In [1]:
from functions.Pre_processing_Casos_Graves import *
from functions.Random_forest import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection._search import GridSearchCV
from sklearn.model_selection._split import StratifiedKFold
from sklearn import svm

from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif, SelectFdr

import optuna
from sklearn.metrics import f1_score, precision_score, recall_score

C:\Users\gabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
def objective(trial):

        classifier_name = trial.suggest_categorical("classifier", ["RandomForest"])

        bootstrap = trial.suggest_categorical("bootstrap", [True, False])
        #max_depth = trial.suggest_int("max_depth", 10, 100)
        criterion = trial.suggest_categorical("criterion", ['gini','entropy'])
        max_features = trial.suggest_categorical("max_features", ['auto', 'sqrt','log2'])
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 4)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
        n_estimators = trial.suggest_int("n_estimators", 100, 2000)

        classifier_obj = RandomForestClassifier(
            bootstrap=bootstrap, max_depth=None,
            criterion=criterion, max_features=max_features,
            min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split,
            n_estimators=n_estimators, random_state=42, n_jobs=-1)

        score = cross_val_score(classifier_obj, X_train, y_train, n_jobs=-1, cv=5,scoring='f1')
        f1 = score.mean()
        return f1

In [2]:
# casos_graves = Pre_Processing_Casos_Graves()
# casos_graves.run(columns_symptoms,columns_to_drop,columns_dieases,path)

In [3]:
df = pd.read_csv(r'C:\Users\gabri\Desktop\Git - Tcc\Estudo_dados_de_covid\Data\Casos Graves\df_casos_graves')

In [4]:
random_forest = Random_forest(df)

In [5]:
random_forest.run_graves()

In [13]:
df = random_forest.df

In [20]:
list_features = ["SYMPTOM_COUGH",'SYMPTOM_COLD','SYMPTOM_AIR_INSUFFICIENCY','SYMPTOM_FEVER','SYMPTOM_LOW_OXYGEN_SATURATION','SYMPTOM_BREATHING_CONDITION','SYMPTOM_TORACIC_APERTURE','SYMPTOM_THROAT_CONDITION',
                'SYMPTOM_HEADACHE','SYMPTOM_BODY_PAIN','SYMPTOM_DIARRHEA','SYMPTOM_RUNNY_NOSE','SYMPTOM_NOSE_CONGESTION','SYMPTOM_WEAKNESS','SYMPTOM_ANOSMIA_OR_HYPOSMIA','SYMPTOM_NAUSEA','SYMPTOM_LACK_OF_APPETITE',
                'SYMPTOM_ABDOMINAL_PAIN','SYMPTOM_CONSCIOUSNESS_DEGRADATION','DISEASE_HEART_OR_VASCULAR','DISEASE_DIABETES','DISEASE_HYPERTENSION','DISEASE_RESPIRATORY','DISEASE_OBESITY','DISEASE_KIDNEY','DISEASE_IMMUNOSUPPRESSION',
                'DISEASE_TABAGIST','DISEASE_ETILISM','DISEASE_LIVER','DISEASE_NEUROLOGIC','death', 'idade', 'sexo_Feminino', 'sexo_Ignorado','sexo_Masculino']

In [21]:
df = df[list_features]
df

,SYMPTOM_COUGH,SYMPTOM_COLD,SYMPTOM_AIR_INSUFFICIENCY,SYMPTOM_FEVER,SYMPTOM_LOW_OXYGEN_SATURATION,SYMPTOM_BREATHING_CONDITION,SYMPTOM_TORACIC_APERTURE,SYMPTOM_THROAT_CONDITION,SYMPTOM_HEADACHE,SYMPTOM_BODY_PAIN,...,DISEASE_IMMUNOSUPPRESSION,DISEASE_TABAGIST,DISEASE_ETILISM,DISEASE_LIVER,DISEASE_NEUROLOGIC,death,idade,sexo_Feminino,sexo_Ignorado,sexo_Masculino
0,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0.460870,0,0,1
1,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.000000,1,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.747826,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.573913,0,0,1
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.591304,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36430,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.669565,0,0,1
36431,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.643478,0,0,1
36432,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.643478,0,0,1
36433,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.634783,1,0,0


In [22]:
X = df.drop("death",axis=1)
y = df['death']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50)
    print(study.best_trial)

[I 2022-08-31 16:25:02,361] A new study created in memory with name: no-name-19e53208-6e5e-44f0-b027-d85734ef13e6
[I 2022-08-31 16:25:52,969] Trial 0 finished with value: 0.3927469081789179 and parameters: {'classifier': 'RandomForest', 'bootstrap': False, 'criterion': 'entropy', 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 6, 'n_estimators': 1780}. Best is trial 0 with value: 0.3927469081789179.
[I 2022-08-31 16:25:59,336] Trial 1 finished with value: 0.36346317339870154 and parameters: {'classifier': 'RandomForest', 'bootstrap': False, 'criterion': 'entropy', 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 279}. Best is trial 0 with value: 0.3927469081789179.
[I 2022-08-31 16:26:22,264] Trial 2 finished with value: 0.3414629832195003 and parameters: {'classifier': 'RandomForest', 'bootstrap': False, 'criterion': 'entropy', 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 3, 'n_estimators': 1257}. Best is

FrozenTrial(number=45, values=[0.404500594479674], datetime_start=datetime.datetime(2022, 8, 31, 16, 49, 40, 576669), datetime_complete=datetime.datetime(2022, 8, 31, 16, 50, 19, 649484), params={'classifier': 'RandomForest', 'bootstrap': True, 'criterion': 'entropy', 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 1842}, distributions={'classifier': CategoricalDistribution(choices=('RandomForest',)), 'bootstrap': CategoricalDistribution(choices=(True, False)), 'criterion': CategoricalDistribution(choices=('gini', 'entropy')), 'max_features': CategoricalDistribution(choices=('auto', 'sqrt', 'log2')), 'min_samples_leaf': IntUniformDistribution(high=4, low=1, step=1), 'min_samples_split': IntUniformDistribution(high=10, low=2, step=1), 'n_estimators': IntUniformDistribution(high=2000, low=100, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=45, state=TrialState.COMPLETE, value=None)


In [25]:
best_params={'bootstrap': True, 'criterion': 'entropy', 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 1842}
best_rf = RandomForestClassifier(**best_params)
scores = ['f1','precision','recall']
best_rf_results_cv = cross_validate(best_rf, X_train, y_train, cv=10,scoring=scores)

for metric in best_rf_results_cv:
    print(f'Média de {metric}: {best_rf_results_cv[metric].mean()}')

C:\Users\gabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\gabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\gabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-p

Média de fit_time: 31.610836815834045
Média de score_time: 1.1915322065353393
Média de test_f1: 0.3976067102650175
Média de test_precision: 0.48969581252218514
Média de test_recall: 0.3348300872548709


In [26]:
best_rf.fit(X_train, y_train)
y_pred = best_rf.predict(X_test)

print('Precisão:',precision_score(y_test,y_pred))
print('Revocação:',recall_score(y_test,y_pred))
print('F1-score:',f1_score(y_test,y_pred))

C:\Users\gabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Precisão: 0.4676079734219269
Revocação: 0.3112216694306247
F1-score: 0.37371390640557584
